In [1]:
#Read csv file
import pandas as pd

df = pd.read_csv('ML_factor_table_file.csv', parse_dates=['date'])

In [79]:
#Seperate unique tickers and dates
tickers = df.ticker.unique()
dates = df.date.unique()

In [80]:
#Seperate the dataset by ticker to a dictionary
tic = df.groupby(['ticker'])

data_by_ticker = {}

for t,d in tic:
    data_by_ticker[t] = d

In [81]:
#Resample the data to monthly and concat to one DataFrame
for t in data_by_ticker:
    data_by_ticker[t] = data_by_ticker[t].set_index(['date']).resample('M').ffill()

whole_df = pd.concat([data_by_ticker[t] for t in data_by_ticker])

In [82]:
#Save the result df to file
whole_df.to_csv("./temp.csv")

# TO DO:
## Select maket cap
## Select factors
## Normalization 
## Target Generation
## Seperate Train/Test
## Link to current codes

# utilities.py

In [27]:
from datetime import datetime
from sklearn import preprocessing
import pandas as pd
from tqdm import tqdm

def MarketCapElimination(df, market_cap = None):
    if not market_cap:
        raise Exception("MarketCapElimination: market_cap must not be None.")
    if not type(market_cap) == int and not type(market_cap) == float:
        raise Exception("MarketCapElimination: market_cap must be a number.")
    
    return df[df.market_cap >= market_cap]

def FactorsSelection(df, factors_list = None):
    if not factors_list:
        print("FactorsSelection: factors_list is None, include all the factors by default.")
        
    if not factors_list or factors_list == 'all':
        return df
    
    factors_list = ['date','ticker'] + factors_list
        
    try:
        df = df[factors_list]
    except KeyError as e:
        raise e
        
    return df

def TrainTestSpliting(df, split_year = None):
    if not split_year:
        raise Exception("TrainTestSpliting: split_year must be specified.")
    if not type(split_year) == int:
        raise Exception("TrainTestSpliting: split_year must be a integer.")
        
    split_year = datetime(year=split_year, month=1, day=1)
    if split_year < datetime.strptime(df.date.min(), '%Y-%m-%d') or split_year >= datetime.strptime(df.date.max(), '%Y-%m-%d'):
        raise Exception("TrainTestSpliting: split_year must be greater than {} and less than {}.".format(df.date.min().year,df.date.max().year))
    
    df_train, df_test = df[df.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d') < split_year)], df[df.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d') >= split_year)]
    
    return (df_train, df_test)
    
def TargetLabelCaculation(df, p = None, period = None):
    if not p:
        print("TargetLabelCaculation: p is None, set p = 10% by default.")
        p = 10
    if not period:
        print("TargetLabelCaculation: period is None, set period = 1 month by default.")
        period = 1
    
    df_by_ticker = {}
    for t, d in df.groupby(['ticker']):
        df_by_ticker[t] = d.sort_values(['date'])
        df_by_ticker[t]['next_return'] = (df_by_ticker[t]['last_price'].shift(-period) - df_by_ticker[t]['last_price']) / df_by_ticker[t]['last_price'] * 100
    
    df = pd.concat([df_by_ticker[t] for t in df_by_ticker])
    df.dropna(axis=0, how='any', inplace=True)
    
    df_by_date = {}
    for date, d in df.groupby(['date']):
        size = d.shape[0]
        sort_next_return = d.sort_values(['next_return'],ascending = False)
        sort_next_return.reset_index(drop=True ,inplace = True)
        """
        top = sort_next_return.head(size*p//100)
        top.loc[:,'target'] = 1
        bottom = sort_next_return.tail(size*p//100)
        bottom.loc[:,'target'] = 0
        df_by_date[date] = pd.concat([top,bottom])
        """
        top = int(size*p/100.)
        tail = size - int(size*p/100.)
        
        sort_next_return.loc[:,'target'] = float('nan')
        sort_next_return.loc[:top,'target'] = 1
        sort_next_return.loc[tail:,'target'] = 0
        
        df_by_date[date] = sort_next_return
        
    df = pd.concat([df_by_date[d] for d in df_by_date])
    
    return df
    
def MinMaxNormalization(df_train, df_test, min_max = None):
    if not min_max:
        print("MinMaxNormalization: min_max is None, set min = 0 and max = 1 by default.")
        min_max = (0,1) 
        
    df_train_key, df_test_key = df_train['ticker'].unique(), df_test['ticker'].unique()
    df_train_by_ticker, df_test_by_ticker = {}, {}
    for t, d in df_train.groupby(['ticker']):
        df_train_by_ticker[t] = d
    for t, d in df_test.groupby(['ticker']):
        df_test_by_ticker[t] = d        
    
    cnt = 0
    for t in df_test_key:
        if t not in df_train_key:
            print('MinMaxNormalization: ticker {} only appear in test set,eliminated.'.format(t))
            df_test_by_ticker.pop(t, None)
            cnt += 1
    print('MinMaxNoramlization: {} tickers are elimnated.'.format(cnt))


    print(len(df_train_by_ticker),len(df_test_by_ticker))

    df_train_normalized_list, df_test_normalized_list = [], []
    for t in tqdm(df_train_by_ticker):
        try:
            df_train_tmp_dict, df_test_tmp_dict = {}, {}
            df_train_tmp_dict['date'] = df_train_by_ticker[t]['date'].tolist(),
            df_train_tmp_dict['ticker'] = df_train_by_ticker[t]['ticker'].tolist()
            df_train_tmp_dict['last_price'] = df_train_by_ticker[t]['last_price'].tolist()
            df_train_tmp_dict['next_return'] = df_train_by_ticker[t]['next_return'].tolist()    
            df_train_tmp_dict['target'] = df_train_by_ticker[t]['target'].tolist()    
        except KeyError as KE:
            raise KE
            
        df_train_by_ticker[t].drop(['date','ticker','last_price','next_return','target'],axis = 1, inplace = True)    
            
        min_max_scaler = preprocessing.MinMaxScaler()
        np_train_scaled = min_max_scaler.fit_transform(df_train_by_ticker[t])
        df_train_normalized = pd.DataFrame(np_train_scaled,columns = df_train_by_ticker[t].columns)
        
        for i in df_train_tmp_dict:
            #To prevent TypeError
            if i == 'date':
                if len(df_train_tmp_dict[i]) == 1:
                    df_train_normalized.loc[:,i] = df_train_tmp_dict[i][0]
                else:
                    df_train_normalized.loc[:,i] = df_train_tmp_dict[i]
            else:
                df_train_normalized.loc[:,i] = df_train_tmp_dict[i]
        
        df_train_normalized_list.append(df_train_normalized)        
        
        try:
            df_test_tmp_dict['date'] = df_test_by_ticker[t]['date'].tolist()
            df_test_tmp_dict['ticker'] = df_test_by_ticker[t]['ticker'].tolist()
            df_test_tmp_dict['last_price'] = df_test_by_ticker[t]['last_price'].tolist()
            df_test_tmp_dict['next_return'] = df_test_by_ticker[t]['next_return'].tolist()
            df_test_tmp_dict['target'] = df_test_by_ticker[t]['target'].tolist()
        except KeyError as KE:
            if t in df_test_by_ticker:
                raise KE
        if t not in df_test_by_ticker:
            continue

        df_test_by_ticker[t].drop(['date','ticker','last_price','next_return','target'],axis = 1, inplace = True)
        
        np_test_scaled = min_max_scaler.transform(df_test_by_ticker[t])
        df_test_normalized = pd.DataFrame(np_test_scaled,columns = df_test_by_ticker[t].columns)
        
        for i in df_test_tmp_dict:
            #To prevent TypeError
            if i == 'date':
                if len(df_test_tmp_dict[i]) == 1:
                    df_test_normalized.loc[:,i] = df_test_tmp_dict[i][0]
                else:
                    df_test_normalized.loc[:,i] = df_test_tmp_dict[i]
            else:
                df_test_normalized.loc[:,i] = df_test_tmp_dict[i]
        
        df_test_normalized_list.append(df_test_normalized)
        
    df_train_min_max_normalized, df_test_min_max_normalized = pd.concat(df_train_normalized_list), pd.concat(df_test_normalized_list)
        
    df_train_min_max_normalized.sort_values(['date'],inplace=True)
    df_test_min_max_normalized.sort_values(['date'],inplace=True)

    return (df_train_min_max_normalized, df_test_min_max_normalized)
    
def EliminateToSeasonal(df, months = None):
    if not months:
        print('EliminateToSeasonal: months is None, choose 1,4,7,10 by default.')
        months = [1,4,7,10]
        
    return df[df.date.apply(lambda x:x.month in months)]

def WriteResultFile(df,predicted_value,f):
    result = pd.DataFrame(df[['date','ticker','next_return']].as_matrix(), columns=['Date','Ticker','Next Return'], copy=True)
    result.loc[:,'Predicted'] = predicted_value
    print('Writing Result to file {}...'.format(f))
    result.to_csv(f, index=False)
    print('Done.')


# MLP.py

In [1]:
from utilities import utilities as util

import math as m
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras import optimizers, regularizers
from keras.constraints import maxnorm

from sklearn.metrics import mean_squared_error

np.random.seed(7)

input_file = 'temp.csv'
output_file = 'temp.csv'

#Read csv file
data = pd.read_csv(input_file)

#Eliminate the stocks which market cap < 100M
data = util.MarketCapElimination(data, 100)

#Select factors
data = util.FactorsSelection(data)

#Generate target label
data = util.TargetLabelCaculation(data)

#Split dataset to train/test
df_train, df_test = util.TrainTestSpliting(data,2016)

#Perform Min Max scaling on each stock
df_train, df_test = util.MinMaxNormalization(df_train, df_test)

#Drop the row which target is NA
df_train.dropna(subset=['target'], inplace=True)
df_test.dropna(subset=['target'], inplace=True)

#Prepare np array for train/test
train_X, train_y = df_train.drop(['date','ticker','last_price','next_return','target'], axis=1).as_matrix(), df_train['target'].as_matrix()
test_X, test_y = df_test.drop(['date','ticker','last_price','next_return','target'], axis=1).as_matrix(), df_test['target'].as_matrix()

#Build the model
model = Sequential()
model.add(Dense(100,kernel_regularizer=regularizers.l2(0.05),batch_input_shape=(len(train_X)/100,train_X.shape[1],)))
#model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(50,kernel_regularizer=regularizers.l2(0.2)))
#model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(300,kernel_regularizer=regularizers.l2(0.1)))
#model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.4))
model.add(Dense(1))
model.add(Activation('sigmoid'))
rmsprop = optimizers.RMSprop(decay=0.0001)
model.compile(loss='binary_crossentropy', optimizer=rmsprop)
model.fit(train_X, train_y, epochs=5, batch_size=len(train_X)/100, verbose=1,shuffle=True)

#Predict the target of test set
predicted_result = model.predict(test_X)
predicted_result = predicted_result[:,0]
                                    
#Cal AE, MSE
abs_error = 0

for i,j in zip(test_y,predicted_result):
    abs_error += np.absolute(i-j)

print "Absolute Error: ",abs_error
print "MSE :",mean_squared_error(test_y,predicted_result)

#Output the predicted value
util.WriteResultFile(df_test,predicted_result,'r.csv')

Using Theano backend.


FactorsSelection: factors_list is None, include all the factors by default.
TargetLabelCaculation: p is None, set p = 10% by default.
TargetLabelCaculation: period is None, set period = 1 month by default.
MinMaxNormalization: min_max is None, set min = 0 and max = 1 by default.
MinMaxNormalization: ticker NMKEF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker GLDLF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker OBNNF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker WLDVF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker CDEV US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker TWIRF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker TKKIF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker GSS US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker MDRPF US Equity only appear in test set,eliminated.
Min

  0%|                                                 | 0/1072 [00:00<?, ?it/s]

test date: 11


  0%|                                         | 1/1072 [00:00<02:23,  7.46it/s]

...

test date: 11


100%|█████████████████████████████████████▉| 1071/1072 [02:42<00:00,  5.39it/s]

test date: 11


100%|██████████████████████████████████████| 1072/1072 [02:42<00:00,  5.40it/s]


Epoch 1/5
2615/2615 [==============================] - ETA: 3s - loss: 28.12 - ETA: 0s - loss: 21.86 - ETA: 0s - loss: 20.41 - ETA: 0s - loss: 16.09 - ETA: 0s - loss: 15.56 - ETA: 0s - loss: 14.46 - ETA: 0s - loss: 13.24 - ETA: 0s - loss: 12.74 - 0s - loss: 11.0828     
Epoch 2/5
2615/2615 [==============================] - ETA: 0s - loss: 2.925 - ETA: 0s - loss: 2.720 - ETA: 0s - loss: 2.349 - ETA: 0s - loss: 1.977 - ETA: 0s - loss: 1.716 - ETA: 0s - loss: 1.533 - ETA: 0s - loss: 1.512 - 0s - loss: 1.2766     
Epoch 3/5
2615/2615 [==============================] - ETA: 0s - loss: 0.701 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.695 - ETA: 0s - loss: 0.695 - ETA: 0s - loss: 0.695 - 0s - loss: 0.6955     
Epoch 4/5
2615/2615 [==============================] - ETA: 2s - loss: 0.693 - ETA: 3s - loss: 0.694 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.694 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.694 - ETA: 1s - loss: 0.694 - ETA: 1s - loss

# Other cell for testing

In [35]:
tmp = df_test.dropna(subset=['target'])

In [2]:
data[data.date=='2015-02-28']

,date,ticker,account_receivable,asset_turnover,basic_eps_5yr_avg_growth,best_target_price,book_value,capital_expenditures,cash_cash_equivalents,cash_flow_per_share,...,resist_level3,high_price,low_price,bbg_mom_1m,bbg_mom_3m,bbg_mom_6m,bbg_mom_12m,bbg_mom_12m_vol_adj,next_return,target
0,2015-02-28,MOGLQ US Equity,0.000000e+00,0.0000,0.0000,0.059,0.000000e+00,0.000,0.0000,0.0000,...,0.00000,0.0150,0.0150,0,0,0,0,0,153.333333,1.0
1,2015-02-28,ZEON US Equity,0.000000e+00,0.0000,0.0000,0.000,0.000000e+00,0.000,0.0000,0.0000,...,0.00000,2.0000,2.0000,0,0,0,0,0,132.500000,1.0
2,2015-02-28,SGQRF US Equity,4.620000e-01,0.0531,-31.1479,0.486,2.842810e+02,-3.649,4.7680,0.0000,...,0.00000,0.4604,0.4180,0,0,0,0,0,60.034752,1.0
3,2015-02-28,MDR US Equity,1.433700e+02,0.7393,-24.5987,4.955,1.488200e+03,-321.187,665.3090,0.5029,...,2.66333,2.5200,2.4300,0,0,0,0,0,53.600000,1.0
4,2015-02-28,SRMLF US Equity,8.205000e+00,0.2365,0.0000,6.233,3.215550e+02,-30.426,0.0000,0.0000,...,0.00000,2.6852,2.6852,0,0,0,0,0,33.137196,1.0
5,2015-02-28,CDXS US Equity,3.870000e+00,0.6602,0.0000,3.267,2.631100e+01,-0.302,26.4870,0.1287,...,3.72800,3.5800,3.4700,0,0,0,0,0,29.178470,1.0
6,2015-02-28,EOPT US Equity,0.000000e+00,0.5500,-451.0570,0.000,-2.053450e+01,0.000,0.0010,-0.0223,...,0.00000,4.0000,4.0000,0,0,0,0,0,27.500000,1.0
7,2015-02-28,KSKGF US Equity,0.000000e+00,0.0000,0.0000,0.555,0.000000e+00,0.000,0.0000,0.0000,...,0.00000,0.5700,0.5700,0,0,0,0,0,27.192982,1.0
8,2015-02-28,KANKF US Equity,1.328400e+05,1.0137,359.2310,5.459,2.869750e+05,0.000,25192.0000,0.0000,...,0.00000,5.7100,5.7100,0,0,0,0,0,26.970228,1.0
9,2015-02-28,SBFFF US Equity,0.000000e+00,0.0000,0.0000,15.423,0.000000e+00,0.000,0.0000,0.0000,...,0.00000,10.3200,10.3200,0,0,0,0,0,23.837209,1.0


In [5]:
df_train[df_train.date=='2015-02-28'].next_return.max()

153.33333333333334

In [6]:
df_test[df_test.date=='2016-05-31']

,account_receivable,asset_turnover,basic_eps_5yr_avg_growth,best_target_price,book_value,capital_expenditures,cash_cash_equivalents,cash_flow_per_share,cash_from_financing,cash_from_investing,...,bbg_mom_1m,bbg_mom_3m,bbg_mom_6m,bbg_mom_12m,bbg_mom_12m_vol_adj,date,last_price,ticker,target,next_return
1,0.268852,0.503704,0.000000,0.044664,0.017443,0.987551,0.077857,0.000000,0.304266,0.919286,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,4.1262,AUY US Equity,1.0,24.402598
1,-0.862746,0.831111,0.000000,-0.513942,-0.128970,1.246916,0.167762,0.334740,-0.411424,1.246392,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,4.3600,NOG US Equity,NaN,5.963303
1,0.255221,0.633200,0.000000,0.809190,1.045454,0.587849,0.263464,0.372778,0.212816,0.514947,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,21.4921,SUM US Equity,NaN,-5.931017
1,-0.916812,-0.195652,0.000000,-0.357523,-0.773392,1.350498,0.448997,1.033136,-0.307169,1.334365,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,12.4500,GLOG US Equity,NaN,4.257028
1,1.336999,0.004695,-0.248906,0.714286,1.138462,1.900757,0.110455,-0.166474,0.695202,1.899833,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,21.5500,UFPT US Equity,NaN,4.593968
1,-0.237565,-0.307656,0.000000,-0.017593,-0.011185,1.040601,0.000000,0.000000,0.245999,1.075223,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,5.5300,GXOCF US Equity,NaN,11.374322
1,0.000000,0.000000,0.000000,0.979058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,5.6000,NUFMF US Equity,NaN,0.000000
1,-0.203323,-0.265647,0.206954,-0.342451,-0.668384,1.026990,0.257276,0.000000,0.611214,1.025415,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,2.6200,CESDF US Equity,NaN,19.465649
1,0.334617,0.597938,0.000000,-0.147889,1.405491,-31.440000,811.004000,1.661600,-139.182000,-35.588000,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,2.3800,ORIG US Equity,NaN,4.201681
1,0.000000,0.000000,0.000000,-0.338462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,2016-05-31,0.5000,BEPTF US Equity,0.0,-19.000000


In [18]:
data.date.tolist()[24000]

'2016-11-30'

In [134]:
test[test.ticker=='STRDF US Equity']

,date,ticker,account_receivable,asset_turnover,basic_eps_5yr_avg_growth,best_target_price,book_value,capital_expenditures,cash_cash_equivalents,cash_flow_per_share,...,resist_level1,resist_level2,resist_level3,high_price,low_price,bbg_mom_1m,bbg_mom_3m,bbg_mom_6m,bbg_mom_12m,bbg_mom_12m_vol_adj
25238,2014-12-31,STRDF US Equity,48.542,0.987,0.0,4.511,154.99,-27.21,-8.132,0.0,...,0.0,0.0,0.0,3.2763,3.2763,0,0,0,0,0
25239,2015-01-31,STRDF US Equity,48.542,0.987,0.0,4.511,154.99,-27.21,-8.132,0.0,...,0.0,0.0,0.0,2.8279,2.8021,0,0,0,0,0


In [10]:
#Read csv file
import pandas as pd

test = pd.read_csv('temp.csv', parse_dates=['date'])

In [11]:
target = TargetLabelCaculation(test,period = 1)
df_train, df_test = TrainTestSpliting(target, 2016)

TargetLabelCaculation: p is None, set p = 10% by default.


In [212]:
df_train, df_test = MinMaxNormalization(df_train, df_test, min_max = None)

MinMaxNormalization: min_max is None, set min = 0 and max = 1 by default.
MinMaxNormalization: ticker FFMGF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker THM US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker VITFF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker RMLRF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker HUMRF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker BKHRF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker GSS US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker GLDLF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker BAA US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker AXU US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker LYDIF US Equity only appear in test set,eliminated.
MinMaxNormalization: ticker CDEV US Equity only appear

21
NKSJF US Equity
train set size: 6
22
LMPRF US Equity
train set size: 13
23
BOALF US Equity
train set size: 13
24
CDSAF US Equity
train set size: 13
25
JIAXF US Equity
train set size: 13
26
SEMG US Equity
train set size: 13
27
STMNF US Equity
train set size: 13
28
BRDBF US Equity
train set size: 13
29
PFIE US Equity
train set size: 3
30
CRTSF US Equity
train set size: 13
31
XJXNF US Equity
train set size: 13
32
TBTEF US Equity
train set size: 9
33
LNGSF US Equity
train set size: 7
34
RBTEF US Equity
train set size: 13
35
OMN US Equity
train set size: 13
36
AGFS US Equity
train set size: 13
37
TAHO US Equity
train set size: 13
38
FUL US Equity
train set size: 13
39
OII US Equity
train set size: 13
40
FANG US Equity
train set size: 13
41
ATHOF US Equity
train set size: 13
42
TTI US Equity
train set size: 13
43
RIC US Equity
train set size: 13
44
PAFRF US Equity
train set size: 13
45
VPNFF US Equity
train set size: 4
46
RGLD US Equity
train set size: 13
47
CVE US Equity
train set size: 

239
IACAF US Equity
train set size: 13
240
SRMLF US Equity
train set size: 13
241
TOLWF US Equity
train set size: 13
242
CPG US Equity
train set size: 13
243
CCC US Equity
train set size: 13
244
CKH US Equity
train set size: 13
245
SU US Equity
train set size: 13
246
HES US Equity
train set size: 13
247
CBT US Equity
train set size: 13
248
NTOIF US Equity
train set size: 13
249
KMG US Equity
train set size: 13
250
MPC US Equity
train set size: 13
251
KUNUF US Equity
train set size: 13
252
VKSC US Equity
train set size: 13
253
PHII US Equity
train set size: 6
254
NBR US Equity
train set size: 13
255
BDREF US Equity
train set size: 13
256
VLPNF US Equity
train set size: 13
257
AVY US Equity
train set size: 13
258
LLPP US Equity
train set size: 2
259
CTXAF US Equity
train set size: 13
260
PTNDF US Equity
train set size: 13
261
VREYF US Equity
train set size: 13
262
YIPCF US Equity
train set size: 13
263
RPM US Equity
train set size: 13
264
GLPEF US Equity
train set size: 13
265
RUBNF US E

454
MRO US Equity
train set size: 13
455
CNMGF US Equity
train set size: 13
456
ATONF US Equity
train set size: 13
457
GRDGF US Equity
train set size: 13
458
ECR US Equity
train set size: 13
459
LPI US Equity
train set size: 13
460
BLL US Equity
train set size: 13
461
HGHGF US Equity
train set size: 13
462
SMFTF US Equity
train set size: 13
463
TREC US Equity
train set size: 13
464
LUPTQ US Equity
train set size: 13
465
SVRGF US Equity
train set size: 13
466
PMGYF US Equity
train set size: 7
467
NHPEF US Equity
train set size: 13
468
BTE US Equity
train set size: 13
469
KDIAF US Equity
train set size: 13
470
LECBF US Equity
train set size: 13
471
GQMNF US Equity
train set size: 13
472
RCKHF US Equity
train set size: 13
473
EPSEF US Equity
train set size: 13
474
STOSF US Equity
train set size: 13
475
SPTJF US Equity
train set size: 13
476
IMDXF US Equity
train set size: 13
477
RNFTF US Equity
train set size: 13
478
FSM US Equity
train set size: 13
479
GZPZF US Equity
train set size: 13


669
LGCLF US Equity
train set size: 13
670
GUYFF US Equity
train set size: 13
671
BZZUF US Equity
train set size: 13
672
KGHPF US Equity
train set size: 13
673
AVD US Equity
train set size: 13
674
REXX US Equity
train set size: 13
675
STLJF US Equity
train set size: 13
676
HAL US Equity
train set size: 13
677
SVNNF US Equity
train set size: 13
678
EPE US Equity
train set size: 13
679
GSM US Equity
train set size: 13
680
KELTF US Equity
train set size: 13
681
ESTE US Equity
train set size: 13
682
NZEOF US Equity
train set size: 13
683
SSAAF US Equity
train set size: 13
684
SPN US Equity
train set size: 13
685
X US Equity
train set size: 13
686
BNEFF US Equity
train set size: 13
687
FARYF US Equity
train set size: 13
688
NWLTF US Equity
train set size: 13
689
IGESF US Equity
train set size: 11
690
THYCF US Equity
train set size: 13
691
LPG US Equity
train set size: 13
692
HLBZF US Equity
train set size: 13
693
RELY US Equity
train set size: 10
694
NISTF US Equity
train set size: 13
695
R

884
CFPUF US Equity
train set size: 13
885
SLB US Equity
train set size: 13
886
KOP US Equity
train set size: 13
887
AKZOF US Equity
train set size: 13
888
XOM US Equity
train set size: 13
889
PTTMF US Equity
train set size: 13
890
NADL US Equity
train set size: 13
891
SDCJF US Equity
train set size: 13
892
ANW US Equity
train set size: 13
893
CPE US Equity
train set size: 13
894
SGSVF US Equity
train set size: 1
895
MLLGF US Equity
train set size: 13
896
ENGGF US Equity
train set size: 13
897
SCEXF US Equity
train set size: 13
898
AETUF US Equity
train set size: 13
899
BALMF US Equity
train set size: 13
900
WIPKF US Equity
train set size: 13
901
AWLCF US Equity
train set size: 13
902
MTGRF US Equity
train set size: 13
903
HRCXF US Equity
train set size: 13
904
STOHF US Equity
train set size: 13
905
HAYN US Equity
train set size: 13
906
SDRL US Equity
train set size: 13
907
OI US Equity
train set size: 13
908
ATW US Equity
train set size: 13
909
ASOLF US Equity
train set size: 13
910
G

In [236]:
EliminateToSeasonal(df_train)

EliminateToSeasonal: months is None, choose 1,4,7,10 by default.


,account_receivable,asset_turnover,basic_eps_5yr_avg_growth,best_target_price,book_value,capital_expenditures,cash_cash_equivalents,cash_flow_per_share,cash_from_financing,cash_from_investing,...,bbg_mom_1m,bbg_mom_3m,bbg_mom_6m,bbg_mom_12m,bbg_mom_12m_vol_adj,date,last_price,ticker,target,next_return
0,0.000000,0.000000,1.0,0.631579,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1140,FOLGF US Equity,NaN,-2.982456
1,0.000000,0.000000,1.0,0.631579,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1106,FOLGF US Equity,NaN,0.000000
2,0.000000,0.000000,1.0,0.631579,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1106,FOLGF US Equity,NaN,0.000000
3,0.000000,1.000000,0.0,0.000000,1.000000,1.000000,1.000000,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1460,FOLGF US Equity,NaN,-4.109589
4,0.000000,1.000000,0.0,0.000000,1.000000,1.000000,1.000000,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1100,FOLGF US Equity,1.0,32.727273
5,0.000000,0.500000,1.0,1.000000,0.213655,1.000000,0.421442,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1100,FOLGF US Equity,1.0,2.181818
6,0.000000,0.500000,1.0,1.000000,0.213655,1.000000,0.421442,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1124,FOLGF US Equity,NaN,1.423488
7,0.000000,0.500000,1.0,1.000000,0.213655,1.000000,0.421442,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1400,FOLGF US Equity,NaN,-21.428571
8,0.000000,0.500000,1.0,1.000000,0.213655,1.000000,0.421442,0.0,0.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,0.1100,FOLGF US Equity,NaN,0.000000
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,2015-10-31,3.2000,VPNFF US Equity,NaN,0.000000
